# Kickstarter
Estimate the succefull projects

### Import Data and Libraries

In [27]:
# Import of relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict

# Reporting
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import classification_report

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Set random seed 
RSEED = 42

# Warning filter
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Loading the kickstarter dataset
df = pd.read_csv('data/kickstarter_projects.csv')

# View the first few rows of the dataset
df.head(10)

,ID,Name,Category,Subcategory,Country,Launched,Deadline,Goal,Pledged,Backers,State
0,1860890148,Grace Jones Does Not Give A F$#% T-Shirt (limi...,Fashion,Fashion,United States,2009-04-21 21:02:48,2009-05-31,1000,625,30,Failed
1,709707365,CRYSTAL ANTLERS UNTITLED MOVIE,Film & Video,Shorts,United States,2009-04-23 00:07:53,2009-07-20,80000,22,3,Failed
2,1703704063,drawing for dollars,Art,Illustration,United States,2009-04-24 21:52:03,2009-05-03,20,35,3,Successful
3,727286,Offline Wikipedia iPhone app,Technology,Software,United States,2009-04-25 17:36:21,2009-07-14,99,145,25,Successful
4,1622952265,Pantshirts,Fashion,Fashion,United States,2009-04-27 14:10:39,2009-05-26,1900,387,10,Failed
5,2089078683,New York Makes a Book!!,Journalism,Journalism,United States,2009-04-28 13:55:41,2009-05-16,3000,3329,110,Successful
6,830477146,Web Site for Short Horror Film,Film & Video,Shorts,United States,2009-04-29 02:04:21,2009-05-29,200,41,3,Failed
7,266044220,Help me write my second novel.,Publishing,Fiction,United States,2009-04-29 02:58:50,2009-05-29,500,563,18,Successful
8,1502297238,Produce a Play (Canceled),Theater,Theater,United States,2009-04-29 04:37:37,2009-06-01,500,0,0,Canceled
9,813230527,Sponsor Dereck Blackburn (Lostwars) Artist in ...,Music,Rock,United States,2009-04-29 05:26:32,2009-05-16,300,15,2,Failed


### Prepare the Data

In [4]:
# View the data types of the columns
df.dtypes

ID              int64
Name           object
Category       object
Subcategory    object
Country        object
Launched       object
Deadline       object
Goal            int64
Pledged         int64
Backers         int64
State          object
dtype: object

In [5]:
# Dropping the unnecessary columns 
df.drop(['ID', 'Pledged', 'Backers',], axis=1, inplace=True)
df.columns

Index(['Name', 'Category', 'Subcategory', 'Country', 'Launched', 'Deadline',
       'Goal', 'State'],
      dtype='object')

In [6]:
# Convert Launched and Deadline columns to datetime
df['Launched'] = pd.to_datetime(df['Launched'])
df['Deadline'] = pd.to_datetime(df['Deadline'])

# Calculate duration in days
df['Duration'] = (df['Deadline'] - df['Launched']).dt.days
df[['Launched', 'Deadline', 'Duration']].head()

,Launched,Deadline,Duration
0,2009-04-21 21:02:48,2009-05-31,39
1,2009-04-23 00:07:53,2009-07-20,87
2,2009-04-24 21:52:03,2009-05-03,8
3,2009-04-25 17:36:21,2009-07-14,79
4,2009-04-27 14:10:39,2009-05-26,28


In [7]:
# Delete all states which are not 'Successful' or 'Failed'
df = df[df['State'].isin(['Successful', 'Failed'])]

In [8]:
# Creating list for categorical features
# and removing the target variable 'State'
cat_features = list(df.columns[df.dtypes==object])
cat_features.remove('State')
cat_features

['Name', 'Category', 'Subcategory', 'Country']

In [9]:
# Creating list for numerical features
num_features = ['Goal', 'Duration'] #list(df.columns[df.dtypes!=object])
num_features

['Goal', 'Duration']

In [10]:
# Create list for datetime features
date_features = ['Launched', 'Deadline']
date_features

['Launched', 'Deadline']

### New Features

In [28]:
# Download the VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')

# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Perform sentiment analysis on the 'Name' column
df['Name_Sentiment'] = df['Name'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Categorize sentiment into positive, negative, or neutral
df['Name_Sentiment_Label'] = df['Name_Sentiment'].apply(
    lambda x: 'Positive' if x > 0.05 else ('Negative' if x < -0.05 else 'Neutral')
)

# Calculate the length of the project name
df['Name_Length'] = df['Name'].apply(len)

# Display the updated dataframe with new features
df[['Name', 'Name_Sentiment', 'Name_Sentiment_Label', 'Name_Length']].head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/thomashubbert/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Name,Name_Sentiment,Name_Sentiment_Label,Name_Length
0,Grace Jones Does Not Give A F$#% T-Shirt (limi...,0.4215,Positive,59
1,CRYSTAL ANTLERS UNTITLED MOVIE,0.0000,Neutral,30
2,drawing for dollars,0.0000,Neutral,19
3,Offline Wikipedia iPhone app,-0.1280,Negative,28
4,Pantshirts,0.0000,Neutral,10


In [24]:
# Define a function to cap outliers
def cap_outliers(series, lower_percentile=0.05, upper_percentile=0.95):
    lower_bound = series.quantile(lower_percentile)
    upper_bound = series.quantile(upper_percentile)
    return series.clip(lower=lower_bound, upper=upper_bound)

# Apply capping to the 'Goal' column
df['Goal'] = cap_outliers(df['Goal'])

# Note: 'Pledged' and 'Backers' columns are not present in the current dataset.
# If they were available, you could apply the same function to those columns.

### Train-Test-Split

Let's split the data set into a training and test set. Using the training set and cross validation we will train our model and find the best hyperparameter combination. In the end the test set will be used for the final evaluation of our best model. 

In [29]:
# Define predictors and target variable
X = df.drop('State', axis=1)
y = df['State']
print(f"We have {X.shape[0]} observations in our dataset and {X.shape[1]} features")
print(f"Our target vector has also {y.shape[0]} values")

We have 331462 observations in our dataset and 11 features
Our target vector has also 331462 values


In [30]:
# Split into train and test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RSEED)

In [31]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (265169, 11)
X_test shape: (66293, 11)
y_train shape: (265169,)
y_test shape: (66293,)


### Pipeline Parts

In [32]:
# Pipeline for categorical features 
cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown='ignore'))
])

In [33]:
# Pipeline for datetime features
date_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

In [34]:
# Pipline for numerical features
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

### Prepare the Pipeline

In [35]:
# Build preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline, cat_features),
        ('date', date_pipeline, date_features)
    ],
    remainder='drop'
)

### Pick a model

In [37]:
# Create a dictionary of models to evaluate
models = {
    'Logistic Regression': LogisticRegression(),
    #'Decision Tree': DecisionTreeClassifier(random_state=RSEED),
    #'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RSEED),
    #'SVM': SVC(probability=True, random_state=RSEED)
    #'KNN': KNeighborsClassifier(n_neighbors=5),
    #'Gradient Boosting': GradientBoostingClassifier(random_state=RSEED),
    #'XGBoost': XGBClassifier(random_state=RSEED),
    #'AdaBoost': AdaBoostClassifier(random_state=RSEED),
    #'Naive Bayes': GaussianNB(),
}

### Run each Model and Evaluate

In [40]:
# Evaluate each model
results = {}
for name, model in models.items():
    # Create pipeline with current model
    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Fit and predict
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    # Store results
    results[name] = accuracy_score(y_test, y_pred)
    
    # Print detailed results
    print(f"\nResults for {name}:")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # Generate confusion matrix
    #print("Confusion Matrix:")
    #print(confusion_matrix(y_test, y_pred))
    
# Compare accuracies
print("\nAccuracy Comparison:")
for name, accuracy in results.items():
    print(f"{name}: {accuracy:.4f}")


Results for Logistic Regression:
Classification Report:
              precision    recall  f1-score   support

      Failed       0.70      0.77      0.74     39509
  Successful       0.61      0.51      0.56     26784

    accuracy                           0.67     66293
   macro avg       0.65      0.64      0.65     66293
weighted avg       0.66      0.67      0.66     66293


Accuracy Comparison:
Logistic Regression: 0.6683


---

### Performance Improvement Plan: 
- Use Jax
- Use Google Cloud
- Reduce to 10% 

Based on your "To Do" list and EDA Data Checking notes, here are some feature engineering ideas that could improve your model quality:

Time-based Features:

Launch Duration: Calculate the duration of the project launch (difference between launch date and deadline).
Pre-Launch vs. Post-Launch Analysis: Create features to capture the time before and after the launch date.
Text-based Features:

Name Sentiment Analysis: Perform sentiment analysis on the project names to extract positive, negative, or neutral sentiments. This could capture the emotional appeal of the project.
Name Length: The length of the project name might be indicative of something.
Category/Subcategory Features:

Combine Categories: Instead of just one-hot encoding, explore creating interaction features between categories and subcategories.
Goal-related Features:

Goal Currency: Extract the currency from the "goal" column (Dollar or Euro) and create a separate feature for it.
Goal Success Ratio: Create a feature that represents the ratio of "pledged" to "goal." This could indicate how close a project was to succeeding.
Backer-related Features:

Backer Pledge Amount: Calculate the average pledge amount per backer (pledged / backers).
Outlier Handling:

Capping Outliers: For "goals," "pledged," and "backers," consider capping outliers to reduce their impact on the model.
To implement these, you'll need to add code to your notebook to perform the feature engineering steps. This might involve:

Converting date columns to datetime objects.
Using string manipulation to extract information from text columns.
Applying mathematical operations to create new features.
Using libraries like nltk or transformers for sentiment analysis.


### ToDo


EDA Data Checking: 
- Data Values
- Data Type 
- Check Distribution of Values
- Missing Values 
- Null Values 
- Correlations
- Data Reduction or Sampling

Checkout Result: 
- Dollar or Euro in GOALS
- Time vs Date in LAUNCHED
- Canceld? in STATE (Target)
- Chancelation Date not there? --> Compare Backers which Cancelled
- GOALS, PLEDGES, BACKERS has Outliers
- SUBCATEGORIE AND CATERGORIE need one hot encoding
- ID needs to be cleaned
- NAME needs some kind of segmentic analysis 
- PreLaunch vs. PostLaunch Analysis  !!

Try for Improvments: 
- Models
- Hyperparamter Tuning 
- Feature Engineering (Segmantic Analyses)